s# Logistic Regression — Full Notes

## 1. What it is
- **Linear classifier** that models conditional probability directly:

  $$
  P(y=1 \mid x) = \sigma(w^\top x + b)
  $$

- **Discriminative model** (models $p(y|x)$ directly), unlike Naïve Bayes or LDA (generative).

---

## 2. The Sigmoid function
$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

- Maps $(-\infty, \infty) \to (0,1)$.
- Nice derivative property:

$$
\frac{d\sigma(z)}{dz} = \sigma(z)\big(1-\sigma(z)\big)
$$

---

## 3. Why not Linear Regression?
- Linear regression: $\hat{y} = w^\top x + b$. Threshold at 0.5 → linear boundary.
- Problems:
  - Predictions not constrained to $[0,1]$.
  - Squared error loss does **not** match Bernoulli distribution assumption.
- Logistic regression:
  - Proper **probabilities**.
  - Uses **log loss**, aligned with classification.

---

## 4. Loss function derivation

For one sample $(x,y)$, model probability is:

$$
P(y|x) = \sigma(z)^y \big(1-\sigma(z)\big)^{1-y}, \quad z = w^\top x + b
$$

### Likelihood for dataset
$$
L(w,b) = \prod_{i=1}^n \sigma(z_i)^{y_i} (1-\sigma(z_i))^{1-y_i}
$$

### Log-likelihood
$$
\ell(w,b) = \sum_{i=1}^n \Big[ y_i \log \sigma(z_i) + (1-y_i)\log (1-\sigma(z_i)) \Big]
$$

### Negative log-likelihood (loss)
$$
J(w,b) = -\ell(w,b)
$$

If averaged:
$$
J(w,b) = -\frac{1}{n}\sum_{i=1}^n \Big[ y_i \log \sigma(z_i) + (1-y_i)\log (1-\sigma(z_i)) \Big]
$$

---

## 5. Gradient derivation (no skipped steps)

### For one sample
$$
J = - \Big[ y \log \sigma(z) + (1-y)\log(1-\sigma(z)) \Big]
$$

Differentiate wrt $w$:

$$
\frac{\partial J}{\partial w}
= - \Bigg[ y \cdot \frac{1}{\sigma(z)} \cdot \frac{\partial \sigma(z)}{\partial w}
+ (1-y) \cdot \frac{1}{1-\sigma(z)} \cdot \frac{\partial (1-\sigma(z))}{\partial w} \Bigg]
$$

Now, compute derivatives:

$$
\frac{\partial \sigma(z)}{\partial w} = \sigma(z)(1-\sigma(z))x
$$

$$
\frac{\partial (1-\sigma(z))}{\partial w} = -\sigma(z)(1-\sigma(z))x
$$

Plug back:

$$
\frac{\partial J}{\partial w}
= - \Big[ y \cdot (1-\sigma(z))x - (1-y)\sigma(z)x \Big]
$$

$$
= (\sigma(z) - y)x
$$

Similarly for bias:

$$
\frac{\partial J}{\partial b} = \sigma(z) - y
$$

---

### Vectorized (all samples)
Let:
- $X \in \mathbb{R}^{n \times d}$ (design matrix),
- $p = \sigma(Xw + b) \in \mathbb{R}^n$,
- $y \in \mathbb{R}^n$.

Then:

$$
\nabla_w J = X^\top (p-y), \qquad
\frac{\partial J}{\partial b} = \mathbf{1}^\top (p-y)
$$

If using mean loss, divide by $n$.

---

## 6. Log-odds interpretation
The model assumes:

$$
\log \frac{P(y=1|x)}{P(y=0|x)} = w^\top x + b
$$

So logistic regression is a **linear model on the log-odds**.

---

## 7. Regularization

### L2 penalty
$$
J = -\ell(w,b) + \frac{\lambda}{2}\|w\|_2^2
$$

Gradient:
$$
\nabla_w J = X^\top (p-y) + \lambda w
$$

Effect: shrinks weights, keeps all features.

---

### L1 penalty
$$
J = -\ell(w,b) + \lambda \|w\|_1
$$

Gradient/subgradient:
$$
\nabla_w J = X^\top (p-y) + \lambda \,\text{sign}(w)
$$

Effect: drives some weights exactly to 0 → **feature selection**.

---

## 8. Closed-form solution?
- **No closed form** for logistic regression.
- Need iterative optimization (gradient descent, Newton’s method, LBFGS, SAGA, etc).
- With L2, Newton’s method = Iteratively Reweighted Least Squares (IRLS).
- With L1, need subgradient or coordinate descent.

---

## 9. Key takeaways
- Same *type* of linear decision boundary as linear regression, but with:
  - Correct **probabilistic interpretation**,
  - Proper **log-loss objective**,
  - Better robustness and generalization.
- Gradient = **(prediction – truth) × input** — simple, elegant.
- Regularization changes gradient directly and affects sparsity/smoothness.


In [15]:
# ML craft: https://www.deep-ml.com/problems/104, https://www.deep-ml.com/problems/106
import numpy as np

def predict_logistic(X: np.ndarray, weights: np.ndarray, bias: float) -> np.ndarray:
	"""
	Implements binary classification prediction using Logistic Regression.

	Args:
		X: Input feature matrix (shape: N x D)
		weights: Model weights (shape: D)
		bias: Model bias

	Returns:
		Binary predictions (0 or 1)
	"""
	z = X @ weights + bias
	return (z >= 0).astype(int)

assert (predict_logistic(np.array([[1, 1], [2, 2], [-1, -1], [-2, -2]]), np.array([1, 1]), 0) == np.array([1, 1, 0, 0])).all()

In [48]:
import numpy as np

def train_logreg(X: np.ndarray, y: np.ndarray, learning_rate: float, iterations: int) -> tuple[list[float], ...]:
    """
    Gradient-descent training algorithm for logistic regression, optimizing parameters with Binary Cross Entropy loss.
    """
    def sigmoid(x: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-x))
    m, n = X.shape
    X = np.hstack((np.ones((m, 1)), X))
    w = np.zeros((n+1, 1))
    y = y.reshape(-1, 1)
    losses = []
    for i in range(iterations):
        y_pred = sigmoid(X @ w)
        log_loss = -(y*np.log(y_pred)+(1-y)*np.log(1-y_pred)).sum()
        losses.append(np.round(log_loss, decimals=4))
        gradient = X.T @ (y_pred - y)
        w = w - learning_rate * gradient
    return w, losses

train_logreg(np.array([[1.0, 0.5], [-0.5, -1.5], [2.0, 1.5], [-2.0, -1.0]]), np.array([1, 0, 1, 0]), 0.01, 20)

(array([[-3.10665949e-04],
        [ 4.03831112e-01],
        [ 3.37881902e-01]]),
 [2.7726,
  2.6485,
  2.533,
  2.4254,
  2.325,
  2.2314,
  2.1441,
  2.0625,
  1.9862,
  1.9148,
  1.8479,
  1.7852,
  1.7263,
  1.6708,
  1.6187,
  1.5696,
  1.5232,
  1.4794,
  1.438,
  1.3988])

# Multinomial (Softmax) Logistic Regression — Full Notes

## 1) Setup & Notation
- Classes: $K \ge 2$.
- Features: $x \in \mathbb{R}^d$.
- Parameters: weight matrix $W \in \mathbb{R}^{d \times K}$, bias $b \in \mathbb{R}^K$.
- Logits:
$$
z = W^\top x + b \in \mathbb{R}^K, \quad z_k = w_k^\top x + b_k
$$
(where $w_k$ is the $k$-th column of $W$).

## 2) Softmax & Class Probabilities
$$
\text{softmax}(z)_k = p_k = \frac{e^{z_k}}{\sum_{j=1}^K e^{z_j}}
$$

Properties:
- $p_k \in (0,1)$ and $\sum_k p_k = 1$.
- Numerical stability uses: $\text{softmax}(z)_k = \frac{e^{z_k - \max_j z_j}}{\sum_j e^{z_j - \max_j z_j}}$.

## 3) Categorical (Multinomial) Likelihood
For one sample with one-hot label $y \in \{0,1\}^K$:
$$
P(y\mid x) = \prod_{k=1}^K p_k^{\,y_k}
$$

Log-likelihood for one sample:
$$
\ell = \sum_{k=1}^K y_k \log p_k
$$

Negative log-likelihood (cross-entropy) for one sample:
$$
J = -\ell = -\sum_{k=1}^K y_k \log p_k
$$

For a dataset $\{(x_i, y_i)\}_{i=1}^n$ (with $Y$ one-hot):
- **Sum loss**:
$$
J(W,b) = -\sum_{i=1}^n \sum_{k=1}^K y_{ik} \log p_{ik}
$$
- **Mean loss** (common in libraries):
$$
J(W,b) = -\frac{1}{n}\sum_{i=1}^n \sum_{k=1}^K y_{ik} \log p_{ik}
$$

## 4) Gradients (No Skipped Steps)

### Key derivatives
- Softmax Jacobian (for one sample):
$$
\frac{\partial p_k}{\partial z_j} = p_k(\delta_{kj} - p_j)
$$

- By chain rule, for one sample’s loss $J = -\sum_k y_k \log p_k$:
  1) $\frac{\partial J}{\partial p_k} = -\frac{y_k}{p_k}$
  2) $\frac{\partial p_k}{\partial z_j} = p_k(\delta_{kj}-p_j)$
  3) So
$$
\frac{\partial J}{\partial z_j}
= \sum_{k=1}^K \frac{\partial J}{\partial p_k}\frac{\partial p_k}{\partial z_j}
= \sum_{k=1}^K \Big(-\frac{y_k}{p_k}\Big) p_k(\delta_{kj}-p_j)
= \sum_{k=1}^K \big(-y_k\delta_{kj} + y_k p_j\big)
= -y_j + p_j \sum_{k=1}^K y_k
= p_j - y_j
$$
(using $\sum_k y_k = 1$ for one-hot).

Thus, for one sample:
$$
\frac{\partial J}{\partial z} = p - y
$$

And since $z = W^\top x + b$:
- For $W$ (each class $k$ column):
$$
\frac{\partial J}{\partial w_k} = (p_k - y_k)\,x
$$
Stacking for all $k$:
$$
\frac{\partial J}{\partial W} = x\, (p - y)^\top \in \mathbb{R}^{d \times K}
$$

- For $b$:
$$
\frac{\partial J}{\partial b} = p - y \in \mathbb{R}^K
$$

### Vectorized over $n$ samples
Let $X \in \mathbb{R}^{n \times d}$, $P \in \mathbb{R}^{n \times K}$, $Y \in \mathbb{R}^{n \times K}$:
- **Sum loss gradients**:
$$
\nabla_W J = X^\top (P - Y), \qquad \nabla_b J = \mathbf{1}^\top (P - Y)
$$
- **Mean loss gradients** (common):
$$
\nabla_W J = \frac{1}{n} X^\top (P - Y), \qquad \nabla_b J = \frac{1}{n}\,\mathbf{1}^\top (P - Y)
$$

## 5) Regularization

### L2 (weight decay)
Add $\frac{\lambda}{2}\|W\|_F^2$ (no penalty on $b$ typically):
$$
J_\text{reg} = J + \frac{\lambda}{2}\|W\|_F^2
$$
Gradient update:
$$
\nabla_W J_\text{reg} = \nabla_W J + \lambda W, \qquad \nabla_b J_\text{reg} = \nabla_b J
$$

### L1
Add $\lambda \|W\|_1$:
$$
\nabla_W J_\text{reg} \in \nabla_W J + \lambda\,\text{sign}(W) \quad (\text{subgradient})
$$
Effect: sparsity across weights.

## 6) OvR vs Multinomial
- **OvR (One-vs-Rest)**: train $K$ independent binary logistic regressions; pick class with largest score/prob.
- **Multinomial (softmax)**: single model with coupled probabilities via softmax.
- In scikit-learn: `LogisticRegression(multi_class="multinomial", solver="lbfgs"|"saga")` trains the true softmax; `multi_class="ovr"` does OvR.

## 7) Numerical Stability Tips
- Use **log-sum-exp** trick for loss:
$$
\log \sum_j e^{z_j} = a + \log \sum_j e^{z_j - a}, \quad a=\max_j z_j
$$
- Stabilized softmax by subtracting $\max(z)$ before `exp`.

## 8) Shapes Recap
- $X:(n\times d),\; W:(d\times K),\; b:(K,),\; Z= XW + \mathbf{1}b^\top:(n\times K)$
- $P=\text{softmax}(Z):(n\times K),\; Y$ one-hot $(n\times K)$
- $\nabla_W J: (d\times K),\; \nabla_b J:(K,)$


In [7]:
# https://www.deep-ml.com/problems/23
# Write a Python function that computes the softmax activation for a given list of scores. The function should return the softmax values as a list, each rounded to four decimal places.
import math

def softmax(scores: list[float]) -> list[float]:
	# Your code here
    exp_lst = [math.exp(s) for s in scores]
    return [round(s/sum(exp_lst), 4) for s in exp_lst]

scores = [1, 2, 3]
output = [0.0900, 0.2447, 0.6652]

In [8]:
softmax(scores)

[0.09, 0.2447, 0.6652]

In [10]:
# https://www.deep-ml.com/problems/39
# In machine learning and statistics, the softmax function is a generalization of the logistic function that converts a vector of scores into probabilities. The log-softmax function is the logarithm of the softmax function, and it is often used for numerical stability when computing the softmax of large numbers.
# Given a 1D numpy array of scores, implement a Python function to compute the log-softmax of the array.
"""
A = np.array([1, 2, 3])
print(log_softmax(A))
array([-2.4076, -1.4076, -0.4076])
"""
import numpy as np

def log_softmax(scores: list) -> np.ndarray:
    exp_lst = np.exp(scores)
    sft_mx = exp_lst/sum(exp_lst)
    lg_sft_mx = np.round(np.log(sft_mx),4)
    return lg_sft_mx
A = np.array([1, 2, 3])
print(log_softmax(A))

[-2.4076 -1.4076 -0.4076]


In [36]:
# https://www.deep-ml.com/problems/105
# Implement a gradient descent-based training algorithm for Softmax regression. Your task is to compute model parameters using Cross Entropy loss and return the optimized coefficients along with collected loss values over iterations. Make sure to round your solution to 4 decimal places
def train_softmaxreg(X: np.ndarray, y: np.ndarray, learning_rate: float, iterations: int) -> tuple[list[float], ...]:
    """
	Gradient-descent training algorithm for Softmax regression, optimizing parameters with Cross Entropy loss.
	"""

    def one_hot_encoding(y: np.ndarray) -> np.ndarray:
        num_classes = len(np.unique(y))
        sorted_classes = np.sort(np.unique(y)).tolist()
        one_hot_encodings = np.zeros((len(y), num_classes))
        for i in range(len(y)):
            one_hot_encodings[i][sorted_classes.index(y[i])] = 1.0
        return one_hot_encodings

    # Step 1 Add bias term to x
    m, n = X.shape
    X = np.hstack((np.ones((m, 1)), X))
    y = one_hot_encoding(y)
    y_m, y_n = y.shape
    w = np.zeros((n+1, y_n))
    losses = []

    def softmax_rows(Z: np.ndarray) -> np.ndarray:
        """Row-wise softmax for a 2D array Z (n×K)."""
        assert Z.ndim == 2, f"Expected 2D, got shape {Z.shape}"
        Zmax = Z.max(axis=1, keepdims=True)          # (n,1)
        expZ = np.exp(Z - Zmax)                      # stability
        return expZ / expZ.sum(axis=1, keepdims=True)

    for i in range(iterations):
        p = softmax_rows(X @ w)
        log_loss = np.round((-np.log(p)*y).sum(), decimals=4)
        losses.append(log_loss)

        gradient = X.T @ (p - y)
        w = w - learning_rate * gradient

    return np.round(w.T, 4).tolist(), losses

train_softmaxreg(np.array([[0.5, -1.2], [-0.3, 1.1], [0.8, -0.6]]), np.array([0, 1, 2]), 0.01, 10)

([[-0.0011, 0.0145, -0.0921],
  [0.002, -0.0598, 0.1263],
  [-0.0009, 0.0453, -0.0342]],
 [3.2958,
  3.2611,
  3.2272,
  3.1941,
  3.1618,
  3.1302,
  3.0993,
  3.0692,
  3.0398,
  3.011])